# 칸딘스키

칸딘스키 모델은 다국어 텍스트-이미지 생성 모델 시리즈입니다. 칸딘스키 2.0 모델은 두 개의 다국어 텍스트 인코더를 사용하고 해당 결과를 UNet에 연결합니다.

[칸딘스키 2.1](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/kandinsky)은 텍스트와 이미지 임베딩 간의 매핑을 생성하기 위해 이미지 사전 모델([`CLIP`](https://huggingface.co/docs/transformers/model_doc/clip))을 포함하도록 아키텍처를 변경합니다. 매핑은 더 나은 텍스트-이미지 정렬을 제공하며 학습 중에 텍스트 임베딩과 함께 사용되어 더 높은 품질의 결과를 생성합니다. 마지막으로 칸딘스키 2.1은 잠재 변수를 이미지로 디코딩하기 위해 [조절 양자화 벡터(MoVQ)](https://huggingface.co/papers/2209.09002) 디코더(사진 사실성을 높이기 위해 공간 조건 정규화 계층을 추가함)를 사용합니다.

[칸딘스키 2.2](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/kandinsky_v22)는 이미지 사전 모델의 이미지 인코더를 더 큰 CLIP-ViT-G 모델로 교체하여 품질을 향상시킴으로써 이전 모델을 개선합니다. 이미지 사전 모델은 또한 다양한 해상도와 종횡비의 이미지에 대해 재학습되어 더 높은 해상도의 이미지와 다양한 이미지 크기를 생성합니다.

[칸딘스키 3](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/kandinsky3)은 아키텍처를 단순화하고 사전 모델과 확산 모델을 포함하는 2단계 생성 프로세스에서 벗어납니다. 대신 칸딘스키 3은 텍스트를 인코딩하기 위해 [Flan-UL2](https://huggingface.co/google/flan-ul2)를 사용하고, [BigGan-deep](https://hf.co/papers/1809.11096) 블록이 있는 UNet과 잠재 변수를 이미지로 디코딩하기 위해 [Sber-MoVQGAN](https://github.com/ai-forever/MoVQGAN)을 사용합니다. 텍스트 이해 및 생성된 이미지 품질은 주로 더 큰 텍스트 인코더와 UNet을 사용하여 달성됩니다.

이 가이드에서는 텍스트-이미지, 이미지-이미지, 인페인팅, 보간 등 다양한 작업에 칸딘스키 모델을 사용하는 방법을 보여줍니다.

시작하기 전에 다음 라이브러리가 설치되어 있는지 확인하십시오.

In [ ]:
# Colab에서 필요한 라이브러리를 설치하려면 주석 처리를 제거하십시오.
#!pip install -q diffusers transformers accelerate

<Tip warning={true}>

칸딘스키 2.1과 2.2의 사용법은 매우 유사합니다! 유일한 차이점은 칸딘스키 2.2가 잠재 변수를 디코딩할 때 `prompt`를 입력으로 받지 않는다는 것입니다. 대신 칸딘스키 2.2는 디코딩 중에 `image_embeds`만 받습니다.

<br>

칸딘스키 3은 더 간결한 아키텍처를 가지고 있으며 사전 모델이 필요하지 않습니다. 즉, [스테이블 디퓨전 XL](https://huggingface.co/docs/diffusers/main/en/using-diffusers/sdxl)과 같은 다른 확산 모델과 사용법이 동일합니다.

</Tip>

## 텍스트-이미지 변환

모든 작업에 대해 Kandinsky 모델을 사용하려면 항상 사전 파이프라인을 설정하여 프롬프트를 인코딩하고 이미지 임베딩을 생성하는 것으로 시작합니다. 사전 파이프라인은 또한 음수 프롬프트 `""`에 해당하는 `negative_image_embeds`를 생성합니다. 더 나은 결과를 얻으려면 사전 파이프라인에 실제 `negative_prompt`를 전달할 수 있지만 이렇게 하면 사전 파이프라인의 유효 배치 크기가 2배로 늘어납니다.

<hfoptions id="text-to-image">
<hfoption id="Kandinsky 2.1">

In [ ]:
from diffusers import KandinskyPriorPipeline, KandinskyPipeline
import torch

prior_pipeline = KandinskyPriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-1-prior", torch_dtype=torch.float16).to("cuda")
pipeline = KandinskyPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16).to("cuda")

prompt = "A alien cheeseburger creature eating itself, claymation, cinematic, moody lighting"
negative_prompt = "low quality, bad quality" # 선택적으로 음수 프롬프트를 포함할 수 있지만 일반적으로 결과가 더 좋습니다.
image_embeds, negative_image_embeds = prior_pipeline(prompt, negative_prompt, guidance_scale=1.0).to_tuple()

이제 모든 프롬프트와 임베딩을 [KandinskyPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky#diffusers.KandinskyPipeline)에 전달하여 이미지를 생성합니다.

In [ ]:
image = pipeline(prompt, image_embeds=image_embeds, negative_prompt=negative_prompt, negative_image_embeds=negative_image_embeds, height=768, width=768).images[0]
image

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinsky-docs/cheeseburger.png"/>
</div>

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
import torch

prior_pipeline = KandinskyV22PriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16).to("cuda")
pipeline = KandinskyV22Pipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16).to("cuda")

prompt = "A alien cheeseburger creature eating itself, claymation, cinematic, moody lighting"
negative_prompt = "low quality, bad quality" # 선택적으로 음수 프롬프트를 포함할 수 있지만 일반적으로 결과가 더 좋습니다.
image_embeds, negative_image_embeds = prior_pipeline(prompt, guidance_scale=1.0).to_tuple()

`image_embeds`와 `negative_image_embeds`를 [KandinskyV22Pipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22Pipeline)에 전달하여 이미지를 생성합니다.

In [ ]:
image = pipeline(image_embeds=image_embeds, negative_image_embeds=negative_image_embeds, height=768, width=768).images[0]
image

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinsky-text-to-image.png"/>
</div>

</hfoption>
<hfoption id="Kandinsky 3">

칸딘스키 3은 사전 모델이 필요하지 않으므로 [Kandinsky3Pipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky3#diffusers.Kandinsky3Pipeline)을 직접 로드하고 프롬프트를 전달하여 이미지를 생성할 수 있습니다.

In [ ]:
from diffusers import Kandinsky3Pipeline
import torch

pipeline = Kandinsky3Pipeline.from_pretrained("kandinsky-community/kandinsky-3", variant="fp16", torch_dtype=torch.float16)
pipeline.enable_model_cpu_offload()

prompt = "A alien cheeseburger creature eating itself, claymation, cinematic, moody lighting"
image = pipeline(prompt).images[0]
image

</hfoption>
</hfoptions>

🤗 Diffusers는 또한 [KandinskyCombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky#diffusers.KandinskyCombinedPipeline) 및 [KandinskyV22CombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22CombinedPipeline)을 사용하여 종단 간 API를 제공하므로 사전 파이프라인과 텍스트-이미지 파이프라인을 별도로 로드할 필요가 없습니다. 결합된 파이프라인은 사전 모델과 디코더를 모두 자동으로 로드합니다. 원하는 경우 `prior_guidance_scale` 및 `prior_num_inference_steps` 매개변수를 사용하여 사전 파이프라인에 대해 다른 값을 설정할 수 있습니다.

[AutoPipelineForText2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForText2Image)를 사용하여 내부적으로 결합된 파이프라인을 자동으로 호출합니다.

<hfoptions id="text-to-image">
<hfoption id="Kandinsky 2.1">

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16)
pipeline.enable_model_cpu_offload()

prompt = "A alien cheeseburger creature eating itself, claymation, cinematic, moody lighting"
negative_prompt = "low quality, bad quality"

image = pipeline(prompt=prompt, negative_prompt=negative_prompt, prior_guidance_scale=1.0, guidance_scale=4.0, height=768, width=768).images[0]
image

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16)
pipeline.enable_model_cpu_offload()

prompt = "A alien cheeseburger creature eating itself, claymation, cinematic, moody lighting"
negative_prompt = "low quality, bad quality"

image = pipeline(prompt=prompt, negative_prompt=negative_prompt, prior_guidance_scale=1.0, guidance_scale=4.0, height=768, width=768).images[0]
image

</hfoption>
</hfoptions>

## 이미지-이미지 변환

이미지-이미지 변환의 경우 초기 이미지와 텍스트 프롬프트를 파이프라인에 전달하여 이미지를 조건화합니다. 사전 파이프라인을 로드하여 시작합니다.

<hfoptions id="image-to-image">
<hfoption id="Kandinsky 2.1">

In [ ]:
import torch
from diffusers import KandinskyImg2ImgPipeline, KandinskyPriorPipeline

prior_pipeline = KandinskyPriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-1-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipeline = KandinskyImg2ImgPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
import torch
from diffusers import KandinskyV22Img2ImgPipeline, KandinskyPriorPipeline

prior_pipeline = KandinskyPriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipeline = KandinskyV22Img2ImgPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

</hfoption>
<hfoption id="Kandinsky 3">

칸딘스키 3은 사전 모델이 필요하지 않으므로 이미지-이미지 파이프라인을 직접 로드할 수 있습니다.

In [ ]:
from diffusers import Kandinsky3Img2ImgPipeline
from diffusers.utils import load_image
import torch

pipeline = Kandinsky3Img2ImgPipeline.from_pretrained("kandinsky-community/kandinsky-3", variant="fp16", torch_dtype=torch.float16)
pipeline.enable_model_cpu_offload()

</hfoption>
</hfoptions>

조건으로 사용할 이미지를 다운로드합니다.

In [ ]:
from diffusers.utils import load_image

# 이미지 다운로드
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
original_image = load_image(url)
original_image = original_image.resize((768, 512))

<div class="flex justify-center">
    <img class="rounded-xl" src="https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"/>
</div>

사전 파이프라인으로 `image_embeds`와 `negative_image_embeds`를 생성합니다.

In [ ]:
prompt = "A fantasy landscape, Cinematic lighting"
negative_prompt = "low quality, bad quality"

image_embeds, negative_image_embeds = prior_pipeline(prompt, negative_prompt).to_tuple()

이제 원본 이미지, 모든 프롬프트 및 임베딩을 파이프라인에 전달하여 이미지를 생성합니다.

<hfoptions id="image-to-image">
<hfoption id="Kandinsky 2.1">

In [ ]:
from diffusers.utils import make_image_grid

image = pipeline(prompt, negative_prompt=negative_prompt, image=original_image, image_embeds=image_embeds, negative_image_embeds=negative_image_embeds, height=768, width=768, strength=0.3).images[0]
make_image_grid([original_image.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinsky-docs/img2img_fantasyland.png"/>
</div>

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
from diffusers.utils import make_image_grid

image = pipeline(image=original_image, image_embeds=image_embeds, negative_image_embeds=negative_image_embeds, height=768, width=768, strength=0.3).images[0]
make_image_grid([original_image.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinsky-image-to-image.png"/>
</div>

</hfoption>
<hfoption id="Kandinsky 3">

In [ ]:
image = pipeline(prompt, negative_prompt=negative_prompt, image=image, strength=0.75, num_inference_steps=25).images[0]
image

</hfoption>
</hfoptions>

🤗 Diffusers는 또한 [KandinskyImg2ImgCombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky#diffusers.KandinskyImg2ImgCombinedPipeline) 및 [KandinskyV22Img2ImgCombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22Img2ImgCombinedPipeline)을 사용하여 종단 간 API를 제공하므로 사전 파이프라인과 이미지-이미지 파이프라인을 별도로 로드할 필요가 없습니다. 결합된 파이프라인은 사전 모델과 디코더를 모두 자동으로 로드합니다. 원하는 경우 `prior_guidance_scale` 및 `prior_num_inference_steps` 매개변수를 사용하여 사전 파이프라인에 대해 다른 값을 설정할 수 있습니다.

[AutoPipelineForImage2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForImage2Image)를 사용하여 내부적으로 결합된 파이프라인을 자동으로 호출합니다.

<hfoptions id="image-to-image">
<hfoption id="Kandinsky 2.1">

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image
import torch

pipeline = AutoPipelineForImage2Image.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16, use_safetensors=True)
pipeline.enable_model_cpu_offload()

prompt = "A fantasy landscape, Cinematic lighting"
negative_prompt = "low quality, bad quality"

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
original_image = load_image(url)

original_image.thumbnail((768, 768))

image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=original_image, strength=0.3).images[0]
make_image_grid([original_image.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image
import torch

pipeline = AutoPipelineForImage2Image.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16)
pipeline.enable_model_cpu_offload()

prompt = "A fantasy landscape, Cinematic lighting"
negative_prompt = "low quality, bad quality"

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
original_image = load_image(url)

original_image.thumbnail((768, 768))

image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=original_image, strength=0.3).images[0]
make_image_grid([original_image.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

</hfoption>
</hfoptions>

## 인페인팅

<Tip warning={true}>

⚠️ 칸딘스키 모델은 이제 검은색 픽셀 대신 ⬜️ **흰색 픽셀**을 사용하여 마스크된 영역을 나타냅니다. 프로덕션에서 [KandinskyInpaintPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky#diffusers.KandinskyInpaintPipeline)을 사용하는 경우 마스크를 흰색 픽셀을 사용하도록 변경해야 합니다.

In [ ]:
# PIL 입력용
import PIL.ImageOps
mask = PIL.ImageOps.invert(mask)

# PyTorch 및 NumPy 입력용
mask = 1 - mask

</Tip>

인페인팅의 경우 원본 이미지, 원본 이미지에서 교체할 영역의 마스크, 인페인트할 내용에 대한 텍스트 프롬프트가 필요합니다. 사전 파이프라인을 로드합니다.

<hfoptions id="inpaint">
<hfoption id="Kandinsky 2.1">

In [ ]:
from diffusers import KandinskyInpaintPipeline, KandinskyPriorPipeline
from diffusers.utils import load_image, make_image_grid
import torch
import numpy as np
from PIL import Image

prior_pipeline = KandinskyPriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-1-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipeline = KandinskyInpaintPipeline.from_pretrained("kandinsky-community/kandinsky-2-1-inpaint", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
from diffusers import KandinskyV22InpaintPipeline, KandinskyV22PriorPipeline
from diffusers.utils import load_image, make_image_grid
import torch
import numpy as np
from PIL import Image

prior_pipeline = KandinskyV22PriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipeline = KandinskyV22InpaintPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

</hfoption>
</hfoptions>

초기 이미지를 로드하고 마스크를 만듭니다.

In [ ]:
init_image = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/cat.png")
mask = np.zeros((768, 768), dtype=np.float32)
# 고양이 머리 위 영역 마스크
mask[:250, 250:-250] = 1

사전 파이프라인으로 임베딩을 생성합니다.

In [ ]:
prompt = "a hat"
prior_output = prior_pipeline(prompt)

이제 초기 이미지, 마스크, 프롬프트 및 임베딩을 파이프라인에 전달하여 이미지를 생성합니다.

<hfoptions id="inpaint">
<hfoption id="Kandinsky 2.1">

In [ ]:
output_image = pipeline(prompt, image=init_image, mask_image=mask, **prior_output, height=768, width=768, num_inference_steps=150).images[0]
mask = Image.fromarray((mask*255).astype('uint8'), 'L')
make_image_grid([init_image, mask, output_image], rows=1, cols=3)

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinsky-docs/inpaint_cat_hat.png"/>
</div>

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
output_image = pipeline(image=init_image, mask_image=mask, **prior_output, height=768, width=768, num_inference_steps=150).images[0]
mask = Image.fromarray((mask*255).astype('uint8'), 'L')
make_image_grid([init_image, mask, output_image], rows=1, cols=3)

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinskyv22-inpaint.png"/>
</div>

</hfoption>
</hfoptions>

또한 종단 간 [KandinskyInpaintCombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky#diffusers.KandinskyInpaintCombinedPipeline) 및 [KandinskyV22InpaintCombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22InpaintCombinedPipeline)을 사용하여 사전 파이프라인과 디코더 파이프라인을 내부적으로 함께 호출할 수 있습니다. 이를 위해 [AutoPipelineForInpainting](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForInpainting)을 사용합니다.

<hfoptions id="inpaint">
<hfoption id="Kandinsky 2.1">

In [ ]:
import torch
import numpy as np
from PIL import Image
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipe = AutoPipelineForInpainting.from_pretrained("kandinsky-community/kandinsky-2-1-inpaint", torch_dtype=torch.float16)
pipe.enable_model_cpu_offload()

init_image = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/cat.png")
mask = np.zeros((768, 768), dtype=np.float32)
# 고양이 머리 위 영역 마스크
mask[:250, 250:-250] = 1
prompt = "a hat"

output_image = pipe(prompt=prompt, image=init_image, mask_image=mask).images[0]
mask = Image.fromarray((mask*255).astype('uint8'), 'L')
make_image_grid([init_image, mask, output_image], rows=1, cols=3)

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
import torch
import numpy as np
from PIL import Image
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipe = AutoPipelineForInpainting.from_pretrained("kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16)
pipe.enable_model_cpu_offload()

init_image = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/cat.png")
mask = np.zeros((768, 768), dtype=np.float32)
# 고양이 머리 위 영역 마스크
mask[:250, 250:-250] = 1
prompt = "a hat"

output_image = pipe(prompt=prompt, image=original_image, mask_image=mask).images[0]
mask = Image.fromarray((mask*255).astype('uint8'), 'L')
make_image_grid([init_image, mask, output_image], rows=1, cols=3)

</hfoption>
</hfoptions>

## 보간

보간을 사용하면 이미지와 텍스트 임베딩 사이의 잠재 공간을 탐색할 수 있으며, 이는 사전 모델의 중간 출력을 볼 수 있는 멋진 방법입니다. 사전 파이프라인과 보간할 두 이미지를 로드합니다.

<hfoptions id="interpolate">
<hfoption id="Kandinsky 2.1">

In [ ]:
from diffusers import KandinskyPriorPipeline, KandinskyPipeline
from diffusers.utils import load_image, make_image_grid
import torch

prior_pipeline = KandinskyPriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-1-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
img_1 = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/cat.png")
img_2 = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/starry_night.jpeg")
make_image_grid([img_1.resize((512,512)), img_2.resize((512,512))], rows=1, cols=2)

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
from diffusers.utils import load_image, make_image_grid
import torch

prior_pipeline = KandinskyV22PriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
img_1 = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/cat.png")
img_2 = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/starry_night.jpeg")
make_image_grid([img_1.resize((512,512)), img_2.resize((512,512))], rows=1, cols=2)

</hfoption>
</hfoptions>

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/cat.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">고양이</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinsky/starry_night.jpeg"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">반 고흐의 별이 빛나는 밤 그림</figcaption>
  </div>
</div>

보간할 텍스트 또는 이미지를 지정하고 각 텍스트 또는 이미지에 대한 가중치를 설정합니다. 가중치가 보간에 미치는 영향을 확인하기 위해 가중치를 실험해 보세요!

In [ ]:
images_texts = ["a cat", img_1, img_2]
weights = [0.3, 0.3, 0.4]

`interpolate` 함수를 호출하여 임베딩을 생성한 다음 파이프라인에 전달하여 이미지를 생성합니다.

<hfoptions id="interpolate">
<hfoption id="Kandinsky 2.1">

In [ ]:
# 프롬프트는 비워 둘 수 있습니다.
prompt = ""
prior_out = prior_pipeline.interpolate(images_texts, weights)

pipeline = KandinskyPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

image = pipeline(prompt, **prior_out, height=768, width=768).images[0]
image

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinsky-docs/starry_cat.png"/>
</div>

</hfoption>
<hfoption id="Kandinsky 2.2">

In [ ]:
# 프롬프트는 비워 둘 수 있습니다.
prompt = ""
prior_out = prior_pipeline.interpolate(images_texts, weights)

pipeline = KandinskyV22Pipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

image = pipeline(prompt, **prior_out, height=768, width=768).images[0]
image

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/kandinskyv22-interpolate.png"/>
</div>

</hfoption>
</hfoptions>

## ControlNet

<Tip warning={true}>

⚠️ ControlNet은 칸딘스키 2.2에서만 지원됩니다!

</Tip>

ControlNet은 깊이 맵이나 가장자리 감지와 같은 추가 입력을 사용하여 대규모 사전 학습된 확산 모델을 조건화할 수 있습니다. 예를 들어, 칸딘스키 2.2를 깊이 맵으로 조건화하여 모델이 깊이 이미지의 구조를 이해하고 보존하도록 할 수 있습니다.

이미지를 로드하고 깊이 맵을 추출해 보겠습니다.

In [ ]:
from diffusers.utils import load_image

img = load_image(
    "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/cat.png"
).resize((768, 768))
img

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/cat.png"/>
</div>

그런 다음 🤗 Transformers의 `depth-estimation` [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline)을 사용하여 이미지를 처리하고 깊이 맵을 검색할 수 있습니다.

In [ ]:
import torch
import numpy as np

from transformers import pipeline

def make_hint(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    hint = detected_map.permute(2, 0, 1)
    return hint

depth_estimator = pipeline("depth-estimation")
hint = make_hint(img, depth_estimator).unsqueeze(0).half().to("cuda")

### 텍스트-이미지 변환 [[controlnet-text-to-image]]

사전 파이프라인과 [KandinskyV22ControlnetPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22ControlnetPipeline)을 로드합니다.

In [ ]:
from diffusers import KandinskyV22PriorPipeline, KandinskyV22ControlnetPipeline

prior_pipeline = KandinskyV22PriorPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

pipeline = KandinskyV22ControlnetPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-controlnet-depth", torch_dtype=torch.float16
).to("cuda")

프롬프트와 부정 프롬프트에서 이미지 임베딩을 생성합니다.

In [ ]:
prompt = "A robot, 4k photo"
negative_prior_prompt = "lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature"

generator = torch.Generator(device="cuda").manual_seed(43)

image_emb, zero_image_emb = prior_pipeline(
    prompt=prompt, negative_prompt=negative_prior_prompt, generator=generator
).to_tuple()

마지막으로 이미지 임베딩과 깊이 이미지를 [KandinskyV22ControlnetPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22ControlnetPipeline)에 전달하여 이미지를 생성합니다.

In [ ]:
image = pipeline(image_embeds=image_emb, negative_image_embeds=zero_image_emb, hint=hint, num_inference_steps=50, generator=generator, height=768, width=768).images[0]
image

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/robot_cat_text2img.png"/>
</div>

### 이미지-이미지 변환 [[controlnet-image-to-image]]

ControlNet을 사용한 이미지-이미지 변환의 경우 다음을 사용해야 합니다.

- 텍스트 프롬프트와 이미지에서 이미지 임베딩을 생성하기 위한 [KandinskyV22PriorEmb2EmbPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22PriorEmb2EmbPipeline)
- 초기 이미지와 이미지 임베딩에서 이미지를 생성하기 위한 [KandinskyV22ControlnetImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22ControlnetImg2ImgPipeline)

🤗 Transformers의 `depth-estimation` [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline)을 사용하여 고양이의 초기 이미지의 깊이 맵을 처리하고 추출합니다.

In [ ]:
import torch
import numpy as np

from diffusers import KandinskyV22PriorEmb2EmbPipeline, KandinskyV22ControlnetImg2ImgPipeline
from diffusers.utils import load_image
from transformers import pipeline

img = load_image(
    "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/cat.png"
).resize((768, 768))

def make_hint(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    hint = detected_map.permute(2, 0, 1)
    return hint

depth_estimator = pipeline("depth-estimation")
hint = make_hint(img, depth_estimator).unsqueeze(0).half().to("cuda")

사전 파이프라인과 [KandinskyV22ControlnetImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22ControlnetImg2ImgPipeline)을 로드합니다.

In [ ]:
prior_pipeline = KandinskyV22PriorEmb2EmbPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

pipeline = KandinskyV22ControlnetImg2ImgPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-controlnet-depth", torch_dtype=torch.float16
).to("cuda")

텍스트 프롬프트와 초기 이미지를 사전 파이프라인에 전달하여 이미지 임베딩을 생성합니다.

In [ ]:
prompt = "A robot, 4k photo"
negative_prior_prompt = "lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature"

generator = torch.Generator(device="cuda").manual_seed(43)

img_emb = prior_pipeline(prompt=prompt, image=img, strength=0.85, generator=generator)
negative_emb = prior_pipeline(prompt=negative_prior_prompt, image=img, strength=1, generator=generator)

이제 [KandinskyV22ControlnetImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22ControlnetImg2ImgPipeline)을 실행하여 초기 이미지와 이미지 임베딩에서 이미지를 생성할 수 있습니다.

In [ ]:
image = pipeline(image=img, strength=0.5, image_embeds=img_emb.image_embeds, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=50, generator=generator, height=768, width=768).images[0]
make_image_grid([img.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/robot_cat.png"/>
</div>

## 최적화

칸딘스키는 매핑을 생성하기 위한 사전 파이프라인과 잠재 변수를 이미지로 디코딩하기 위한 두 번째 파이프라인이 필요하기 때문에 독특합니다. 최적화 노력은 계산의 대부분이 수행되는 두 번째 파이프라인에 집중해야 합니다. 다음은 추론 중에 칸딘스키를 개선하기 위한 몇 가지 팁입니다.

1. PyTorch < 2.0을 사용하는 경우 [xFormers](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/xformers)를 활성화합니다.

```diff
  from diffusers import DiffusionPipeline
  import torch

  pipe = DiffusionPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16)
+ pipe.enable_xformers_memory_efficient_attention()
```

2. PyTorch >= 2.0을 사용하는 경우 `torch.compile`을 활성화하여 스케일드 닷 프로덕트 어텐션(SDPA)을 자동으로 사용합니다.

```diff
  pipe.unet.to(memory_format=torch.channels_last)
+ pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
```

이는 어텐션 프로세서를 [AttnAddedKVProcessor2_0](https://huggingface.co/docs/diffusers/main/en/api/attnprocessor#diffusers.models.attention_processor.AttnAddedKVProcessor2_0)을 사용하도록 명시적으로 설정하는 것과 동일합니다.

In [ ]:
from diffusers.models.attention_processor import AttnAddedKVProcessor2_0

pipe.unet.set_attn_processor(AttnAddedKVProcessor2_0())

3. 메모리 부족 오류를 방지하기 위해 [enable_model_cpu_offload()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_model_cpu_offload)를 사용하여 모델을 CPU로 오프로드합니다.

```diff
  from diffusers import DiffusionPipeline
  import torch

  pipe = DiffusionPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16)
+ pipe.enable_model_cpu_offload()
```

4. 기본적으로 텍스트-이미지 파이프라인은 [DDIMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/ddim#diffusers.DDIMScheduler)를 사용하지만 [DDPMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/ddpm#diffusers.DDPMScheduler)와 같은 다른 스케줄러로 교체하여 추론 속도와 이미지 품질 간의 절충안에 미치는 영향을 확인할 수 있습니다.

In [ ]:
from diffusers import DDPMScheduler
from diffusers import DiffusionPipeline

scheduler = DDPMScheduler.from_pretrained("kandinsky-community/kandinsky-2-1", subfolder="ddpm_scheduler")
pipe = DiffusionPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", scheduler=scheduler, torch_dtype=torch.float16, use_safetensors=True).to("cuda")